In [102]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import utils as mutil

NameError: name 'null' is not defined

In [ ]:
titanic_train = pd.read_csv("titanic/train.csv")
titanic_test = pd.read_csv("titanic/test.csv")
titanic_submission_ex = pd.read_csv("titanic/gender_submission.csv")

In [ ]:
titanic_test

In [ ]:
# We're defining a function missing data to check if the values missing in the differents data_set and then 
#to decide how to handle missing values

def missingdata(data):
    total = data.isnull().sum().sort_values(ascending = False)
    percent = (data.isnull().sum()/data.isnull().count()*100).sort_values(ascending = False)
    ms=pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    ms= ms[ms["Percent"] > 0]
    f,ax =plt.subplots(figsize=(8,6))
    plt.xticks(rotation='90')
    fig=sns.barplot(ms.index, ms["Percent"],color="green",alpha=0.8)
    plt.xlabel('Features', fontsize=15)
    plt.ylabel('Percent of missing values', fontsize=15)
    plt.title('Percent missing data by feature', fontsize=15)
    return ms

In [ ]:
missingdata(titanic_train)


In [ ]:
missingdata(titanic_test)

In [ ]:
titanic_train['Embarked'].fillna(titanic_train['Embarked'].mode()[0], inplace = True) # We remark that there were values for
#the Embarked futures in our titanic_test missing in the titanic_train. So we decide to fill the titanic_train as many values
#were missed.
titanic_test['Fare'].fillna(titanic_test['Fare'].median(), inplace = True)


#Cabin Featueres has more than 75% of missing data in both Test and train data so we are remove the Cabin
drop_column = ['Cabin']
titanic_train.drop(drop_column, axis=1, inplace = True)
titanic_test.drop(drop_column,axis=1,inplace=True) 



In [ ]:
#Both the test and train Age features contains more the 15% of missing Data so we are fill with the median¶
titanic_train['Age'].fillna(titanic_train['Age'].mean(), inplace = True)
titanic_test['Age'].fillna(titanic_test['Age'].mean(), inplace = True)

In [ ]:
# We are now checking if all null/nan values are well replaced.
print('check the nan value in train data')
print(titanic_train.isnull().sum())
print('___'*30)
print('check the nan value in test data')
print(titanic_test.isnull().sum())

In [ ]:
## combine test and train as single to apply some function
all_data=[titanic_train,titanic_test]

In [ ]:
# Create new feature FamilySize as a combination of SibSp and Parch
for dataset in all_data:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1

In [ ]:
import re
# Define function to extract titles from passenger names
def get_title(name):
    title_search = re.search(' ([A-Za-z]+)\.', name)
    # If the title exists, extract and return it.
    if title_search:
        return title_search.group(1)
    return ""
# Create a new feature Title, containing the titles of passenger names
for dataset in all_data:
    dataset['Title'] = dataset['Name'].apply(get_title)
# Group all non-common titles into one single grouping "Rare"
for dataset in all_data:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 
                                                 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')

In [ ]:
titanic_train

In [ ]:
## create a binary for age features
for dataset in all_data:
    dataset['Age_bin'] = pd.cut(dataset['Age'], bins=[0,12,20,50,95,], labels=['Children','Teenage','Adult','Elder'])

In [ ]:
## create a binary for fare features
for dataset in all_data:
    dataset['Fare_bin'] = pd.cut(dataset['Fare'], bins=[0,7.91,14.45,31,120], labels=['Low_fare','Median_fare','Average_fare','High_fare'])

In [ ]:
### for keeping a reference of our numerical analysis, we save a copy of the data
trainset=titanic_train
testset=titanic_test

In [ ]:
all_dat=[trainset,testset]

In [ ]:
trainset.columns
testset.columns

In [ ]:
for dataset in all_dat:
    drop_column = ['Age','Fare','Name','Ticket']
    dataset.drop(drop_column, axis=1, inplace = True)

In [ ]:
# We are dropping PassengerId column because is just a coutner id
drop_column = ['PassengerId']
trainset.drop(drop_column, axis=1, inplace = True)

In [103]:
 #now every thing almost ready only one step we converted the catergical features in numerical by using dummy variable¶

In [104]:
trainset = pd.get_dummies(trainset, columns = ["Sex","Title","Age_bin","Embarked","Fare_bin"],
                             prefix=["Sex","Title","Age_type","Em_type","Fare_type"])

KeyError: "None of [Index(['Sex', 'Title', 'Age_bin', 'Embarked', 'Fare_bin'], dtype='object')] are in the [columns]"

In [ ]:
testset = pd.get_dummies(testset, columns = ["Sex","Title","Age_bin","Embarked","Fare_bin"],
                             prefix=["Sex","Title","Age_type","Em_type","Fare_type"])

In [ ]:
sns.heatmap(trainset.corr(),annot=True,cmap='RdYlGn',linewidths=0.2) #data.corr()-->correlation matrix
fig=plt.gcf()
fig.set_size_inches(20,12)
plt.show()

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split #for split the data
from sklearn.metrics import accuracy_score  #for accuracy_score
from sklearn.model_selection import KFold #for K-fold cross validation
from sklearn.model_selection import cross_val_score #score evaluation
from sklearn.model_selection import cross_val_predict #prediction
from sklearn.metrics import confusion_matrix #for confusion matrix
all_features = trainset.drop("Survived",axis=1)
Targeted_feature = trainset["Survived"]
X_train,X_test,y_train,y_test = train_test_split(all_features,Targeted_feature,test_size=0.3,random_state=42)
X_train.shape,X_test.shape,y_train.shape,y_test.shape

In [ ]:
model = KNeighborsClassifier(n_neighbors = 4)
model.fit(X_train,y_train)
prediction_knn_train=model.predict(X_train)
prediction_knn_test=model.predict(X_test)
print('--------------The Accuracy of the model----------------------------')
print('The accuracy of the K Nearst Neighbors Classifier on trainset is',round(accuracy_score(prediction_knn_train,y_train)*100,2))
print('The accuracy of the K Nearst Neighbors Classifier on testset is',round(accuracy_score(prediction_knn_test,y_test)*100,2))
kfold = KFold(n_splits=10, random_state=22, shuffle=True) # k=10, split the data into 10 equal parts
result_knn=cross_val_score(model,all_features,Targeted_feature,cv=10,scoring='accuracy')
print('The cross validated score for K Nearest Neighbors Classifier is:',round(result_knn.mean()*100,2))
y_pred = cross_val_predict(model,all_features,Targeted_feature,cv=10)

In [ ]:
mutil.plot_roc(model, X_test, y_test title="ROC curve for best fit model without feature engineering (KNN, Titanic)")

In [ ]:
f1 = mutil.calc_f1(model, X_test, y_test)
print(f"Weighted F1-score = {f1}")